## Rat PA Study Data Loading & Preprocessing

The script is designed to handle the preprocessing of data in the Rat PA Study. Its primary purpose is to reorganize the data format for improved usability, without removing any data.

This script will only get the data needed and process them in the data folder.

In [25]:
import data_loading
import os
import pandas as pd

# the path of the raw dataset folder
source_folder = "../../../University of Adelaide/Rat PA Study/"

# the path of the destination dataset folder
destination_folder = "../../data/Rat PA Study/"

# create destination folder if not exists
os.makedirs(destination_folder, exist_ok=True)

### 1. Generating report summary

We will firstly get the summary of the dataset in one spreadsheet, which includes all the data from the study and the 6 parameters (VDP, MSV, TV, VH, VHSS, VHLS).

We will do that by scraping the data from the reports.

Also, we will add the column `FileName` of that animal to extract additional data.

In [26]:
df = data_loading.create_report_summary(source_folder+"PDF_reports/")
df[:5]

,ScanName,DatePrepared,VDP(%),MSV(mL/mL),TV(L),VH(%),VHSS(%),VHLS(%),FileName
0,4627,2023-08-09-01:40:46.817952,16.7,0.119,1.207,51.44,21.59,32.30,4627.WT.PA21.ventilationReport.pdf
1,3549,2021-10-11-03:51:30.887225,14.8,0.170,0.119,49.33,19.63,41.12,3549.WT.PA7.ventilationReport.pdf
2,3557,2021-10-14-19:08:05.562148,18.2,0.180,0.114,52.73,17.75,40.79,3557.KO.PA7.ventilationReport.pdf
3,4322,2023-05-12-01:45:19.221067,13.7,0.151,1.304,43.21,17.45,31.62,4322.WT.PA7.ventilationReport.pdf
4,4580,2023-08-09-01:41:43.754457,15.1,0.200,1.199,43.52,19.30,32.13,4580.WT.PA21.ventilationReport.pdf


Now we will add the columns `Genotype` & `BatchType`, extracted from the `FileName` column.

In [27]:
genotypes = []
batchtypes = []

for file_name in df["FileName"]:
    if len(file_name.split(".")) == 5:
        genotype = file_name.split(".")[1].replace(" ", "")
        batchtype = file_name.split(".")[2].replace(" ", "")
        genotypes.append(genotype)
        batchtypes.append(int(batchtype[2:]))
    else:
        if len(file_name.split(".")) == 4:
            genotype = file_name.split(".")[1].replace(" ", "")
            batchtype = -1
        else:
            genotype = "N/A"
            batchtype = -1
        genotypes.append(genotype)
        batchtypes.append(batchtype)

df["Genotype"] = genotypes
df["BatchType"] = batchtypes

We will drop the column `FileName`, and save this Dataframe as a csv file and store it in the data folder

In [28]:
df = df.drop('FileName', axis=1)
df.to_csv(destination_folder+"report_summary.csv", index=False)
df

,ScanName,DatePrepared,VDP(%),MSV(mL/mL),TV(L),VH(%),VHSS(%),VHLS(%),Genotype,BatchType
0,4627,2023-08-09-01:40:46.817952,16.7,0.119,1.207,51.44,21.59,32.30,WT,21
1,3549,2021-10-11-03:51:30.887225,14.8,0.170,0.119,49.33,19.63,41.12,WT,7
2,3557,2021-10-14-19:08:05.562148,18.2,0.180,0.114,52.73,17.75,40.79,KO,7
3,4322,2023-05-12-01:45:19.221067,13.7,0.151,1.304,43.21,17.45,31.62,WT,7
4,4580,2023-08-09-01:41:43.754457,15.1,0.200,1.199,43.52,19.30,32.13,WT,21
...,...,...,...,...,...,...,...,...,...,...
67,3550,2021-10-10-03:59:08.060981,16.5,0.140,0.120,48.07,20.09,36.23,KO,7
68,3511,2021-10-05-20:37:43.284350,12.8,0.180,0.117,43.17,17.67,33.52,WT,7
69,3952,2023-06-13-13:08:08.556431,9.7,0.125,1.103,36.86,13.21,28.39,N/A,-1
70,4389,2023-02-27-21:57:39.030029,12.0,0.180,1.157,38.54,17.74,27.66,CF,7


### 2. Copying 3D csv files

Now we will move all the csv data from the raw dataset to the destination dataset as well

In [29]:
data_loading.copy_3d_csvs(source_folder+"csv/", destination_folder+"csv/")

### 3. Updating from metadata

We will now update the report summary with the new information from the `metadata_with_some_extras.xlsx` (Sex, Age, Weight)

In [30]:
# Open metadata.xslx and extract necessary info into a dict
df = pd.read_excel(source_folder+"metadata_with_some_extras.xlsx", skiprows=2)
df = df.drop(index=0)
df.reset_index(drop=True, inplace=True)

rat_dict = {}
for i, subject in enumerate(df["Subject"]):
    if df["Exclude"][i] != "Y":
        subject_dict = {}
        subject_dict["sex"] = df["Sex"][i].upper()
        subject_dict["weight"] = df["Weight"][i]
        subject_dict["age"] = df["Age"][i]
        rat_dict[str(subject)] = subject_dict

# Open report summary and make neccessary changes
df = pd.read_csv(destination_folder+"report_summary.csv")

sex = []
weight = []
age = []
for scan_name in df.ScanName:
    if scan_name in list(rat_dict):
        sex.append(rat_dict[scan_name]["sex"])
        weight.append(rat_dict[scan_name]["weight"])
        age.append(rat_dict[scan_name]["age"])
    else:
        sex.append("N/A")
        weight.append(-1)
        age.append(-1)

df["Sex"] = sex
df["Weight"] = weight
df["Age"] = age

# save csv
df.to_csv(destination_folder+"report_summary.csv", index=False)
